In [1]:
import gspread
gc = gspread.service_account(filename='creds/lifts-service.json')
sheet = gc.open('Body Measures')
ws = sheet.get_worksheet(0)  


In [56]:
import pandas as pd
import altair as alt

alt.renderers.enable('mimetype')

def make_chart_LR(L_index: int, R_index: int) -> alt.Chart:
    date = ws.col_values(1)
    L = ws.col_values(L_index)
    R = ws.col_values(R_index)

    title = str(L[0]).split(' ')[0]

    df = pd.DataFrame(
        {
            "date": date[1:],
            "L": L[1:],
            "R": R[1:]
        }
    )

    df["L"] = pd.to_numeric(df["L"])
    df["R"] = pd.to_numeric(df["R"])

    df = df[df['L'] != 0]

    df['date'] = pd.to_datetime(df['date'], dayfirst=True)

    chart = alt.Chart(df).transform_fold(
        ['L', 'R']
    ).mark_line(point = True).encode(
        alt.X('date:T', timeUnit='yearmonthdate', sort='-x', title='date', axis= alt.Axis(
            grid=False)),
        alt.Y('value:Q', scale=alt.Scale(zero=False), title = title + ' Size (cm)', impute=alt.ImputeParams(frame=[-1, 1], method='mean')),
        color='key:N'
    ).add_selection(
        alt.selection(type='single', on='mouseover', fields=['symbol'], nearest=True)
    ).properties(
        width=1000,
    )

    return chart

def make_chart_LR_interactive(L_index: int, R_index: int) -> alt.Chart:
    date = ws.col_values(1)
    L = ws.col_values(L_index)
    R = ws.col_values(R_index)

    title = str(L[0]).split(' ')[0]

    df = pd.DataFrame(
        {
            "date": date[1:],
            "L": L[1:],
            "R": R[1:]
        }
    )

    df["L"] = pd.to_numeric(df["L"])
    df["R"] = pd.to_numeric(df["R"])

    df = df[df['L'] != 0]

    df['date'] = pd.to_datetime(df['date'], dayfirst=True)


    highlight = alt.selection(type='single', on='mouseover',
                          fields=['key'], nearest=True)

    chart = alt.Chart(df).transform_fold(
        ['L', 'R']
    ).mark_line().encode(
        alt.X('date:T', timeUnit='yearmonthdate', sort='-x', title='date', axis= alt.Axis(
            grid=True)),
        alt.Y('value:Q', scale=alt.Scale(zero=False), title = title + ' Size (cm)', impute=alt.ImputeParams(frame=[-1, 1], method='mean')),
        color='key:N',
        size=alt.condition(~highlight, alt.value(1), alt.value(3))
    ).properties(
        width=1000,
    )

    points = alt.Chart(df).transform_fold(
        ['L', 'R']
    ).mark_point(opacity=1, filled=True).encode(
        x='date:T',
        y='value:Q',
        color='key:N',
        shape=alt.Shape('value:Q', scale = alt.Scale(range=["cross"], zero=True)
    ).add_selection(
        highlight
    )

    return chart + points

def make_chart(index: int) -> alt.Chart:
    date = ws.col_values(1)
    val = ws.col_values(index)
    
    title = str(val[0])

    df = pd.DataFrame(
        {
            "date": date[1:],
            title: val[1:]
        }
    )

    df['date'] = pd.to_datetime(df['date'], dayfirst=True)
    df[title] = pd.to_numeric(df[title])
    df = df[df[title] != 0]


    chart = alt.Chart(df).mark_line(point = True).encode(
        alt.X('date:T', timeUnit='yearmonthdate', sort='-x', title='date', axis= alt.Axis(
            grid=False)),
        alt.Y(title + ':Q', scale=alt.Scale(zero=False), title = title, impute=alt.ImputeParams(frame=[-1, 1], method='mean')) 
    ).properties(
        width=1500,
        height=250
    )

    return chart

# The weight column causes JS errors with impute, no clue why 
def make_chart_no_impute(index: int) -> alt.Chart:
    date = ws.col_values(1)
    val = ws.col_values(index)
    
    title = str(val[0])

    df = pd.DataFrame(
        {
            "date": date[1:],
            title: val[1:]
        }
    )

    df['date'] = pd.to_datetime(df['date'], dayfirst=True)
    df[title] = pd.to_numeric(df[title])
    df = df[df[title] != 0]


    chart = alt.Chart(df).mark_line(point = True).encode(
        alt.X('date:T', timeUnit='yearmonthdate', sort='-x', title='date', axis= alt.Axis(
            grid=False)),
        alt.Y(title + ':Q', scale=alt.Scale(zero=False), title = title) 
    ).properties(
        width=1500,
        height=250
    )

    return chart

In [3]:
# weight graph (lower is better)

chart = make_chart_no_impute(4)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [4]:
# Hips
chart = make_chart(9)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [5]:
# Waist (lower is better)
chart = make_chart(10)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [6]:
# Chest

chart = make_chart(11)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [57]:
# Calve Graph
chart = make_chart_LR_interactive(5, 6)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [27]:
# Thighs
chart = make_chart_LR(7,8)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [9]:
# Forearms
chart = make_chart_LR(12,13)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [58]:
# Biceps
chart = make_chart_LR_interactive(14, 15)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html
